# 掛載雲端硬碟

前處理完畢檔案之路徑：

/content/drive/MyDrive/AI CUP/多模態病理嗓音分類競賽/Training Dataset/Preprocessed Data/testData_0505.csv

/content/drive/MyDrive/AI CUP/多模態病理嗓音分類競賽/Training Dataset/Preprocessed Data/trainData_0505.csv

In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/AI CUP/多模態病理嗓音分類競賽')

Mounted at /content/drive


# 載入套件

In [2]:
import numpy as np
import pandas as pd
import librosa

import tensorflow as tf
from tensorflow.keras.layers import Activation, BatchNormalization, Dense, LayerNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics, svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score, roc_curve, auc, accuracy_score
#---#

%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import clear_output

# 載入訓練資料

已切分好train&test(9:1)

In [3]:
# 讀取訓練資料集
# source_df = pd.read_csv('./Training Dataset/trainingData_preprocessed.csv')
training_df = pd.read_csv('./Training Dataset/Preprocessed Data/trainData_0505.csv', names=['Unnamed: 0', 'ID', 'Sex', 'Age', 'Disease_category', 'Narrow_pitch_range',
       'Decreased_volume', 'Fatigue', 'Dryness', 'Lumping', 'heartburn',
       'Choking', 'Eye_dryness', 'PND', 'Smoking', 'PPD', 'Drinking',
       'frequency', 'Onset_of_dysphonia ', 'Noise_at_work',
       'Occupational_vocal_demand', 'Diabetes', 'Hypertension', 'CAD',
       'Head_and_Neck_Cancer', 'Head injury', 'CVA',
       'Voice_handicap_index_-_10', 'Diurnal_pattern_1', 'Diurnal_pattern_2',
       'Diurnal_pattern_3', 'Diurnal_pattern_4'])
test_df = pd.read_csv('./Training Dataset/Preprocessed Data/testData_0505.csv', names=['Unnamed: 0', 'ID', 'Sex', 'Age', 'Disease_category', 'Narrow_pitch_range',
       'Decreased_volume', 'Fatigue', 'Dryness', 'Lumping', 'heartburn',
       'Choking', 'Eye_dryness', 'PND', 'Smoking', 'PPD', 'Drinking',
       'frequency', 'Onset_of_dysphonia ', 'Noise_at_work',
       'Occupational_vocal_demand', 'Diabetes', 'Hypertension', 'CAD',
       'Head_and_Neck_Cancer', 'Head injury', 'CVA',
       'Voice_handicap_index_-_10', 'Diurnal_pattern_1', 'Diurnal_pattern_2',
       'Diurnal_pattern_3', 'Diurnal_pattern_4'])
# 刪除索引
# source_df=source_df.drop(['Unnamed: 0','ID'],axis=1)
# training_df=training_df.drop(['ID'],axis=1)
# test_df=test_df.drop(['ID'],axis=1)
# 顯示資訊
# print('source_df.shape :', source_df.shape)
# print('source_df.columns :', source_df.columns)
print('training_df.shape :', training_df.shape)
print('training_df.columns :', training_df.columns)
print('------')
print('test_df.shape :', test_df.shape)
print('test_df.columns :', test_df.columns)

training_df.shape : (901, 32)
training_df.columns : Index(['Unnamed: 0', 'ID', 'Sex', 'Age', 'Disease_category',
       'Narrow_pitch_range', 'Decreased_volume', 'Fatigue', 'Dryness',
       'Lumping', 'heartburn', 'Choking', 'Eye_dryness', 'PND', 'Smoking',
       'PPD', 'Drinking', 'frequency', 'Onset_of_dysphonia ', 'Noise_at_work',
       'Occupational_vocal_demand', 'Diabetes', 'Hypertension', 'CAD',
       'Head_and_Neck_Cancer', 'Head injury', 'CVA',
       'Voice_handicap_index_-_10', 'Diurnal_pattern_1', 'Diurnal_pattern_2',
       'Diurnal_pattern_3', 'Diurnal_pattern_4'],
      dtype='object')
------
test_df.shape : (101, 32)
test_df.columns : Index(['Unnamed: 0', 'ID', 'Sex', 'Age', 'Disease_category',
       'Narrow_pitch_range', 'Decreased_volume', 'Fatigue', 'Dryness',
       'Lumping', 'heartburn', 'Choking', 'Eye_dryness', 'PND', 'Smoking',
       'PPD', 'Drinking', 'frequency', 'Onset_of_dysphonia ', 'Noise_at_work',
       'Occupational_vocal_demand', 'Diabetes', 'Hy

# 切分訓練集與測試集

In [4]:
# 訓練資料標記轉換
training_data = training_df.drop(['Voice_handicap_index_-_10'], axis=1) # 刪除VHI-10嗓音障礙指標
print('training_data.shape :', training_data.shape)
for tr in range(len(training_data['Disease_category'])-1):
  training_data.Disease_category[tr+1]=int(float(training_data.Disease_category[tr+1]))-1

training_data.shape : (901, 31)


<ipython-input-4-4ef174afb371>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.Disease_category[tr+1]=int(float(training_data.Disease_category[tr+1]))-1


In [5]:
# 驗證資料標記轉換
test_data = test_df.drop(['Voice_handicap_index_-_10'], axis=1) # 刪除VHI-10嗓音障礙指標
print('test_data.shape :', test_data.shape)
for tr in range(len(test_data['Disease_category'])-1):
  test_data.Disease_category[tr+1]=int(float(test_data.Disease_category[tr+1]))-1

test_data.shape : (101, 31)


<ipython-input-5-0e811c32b2a3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.Disease_category[tr+1]=int(float(test_data.Disease_category[tr+1]))-1


# 模型訓練

參考：

https://towardsdatascience.com/multi-class-classification-with-transformers-6cf7b59a033a

https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html?source=post_page

https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_multiclass_classification.ipynb#scrollTo=iNCaZ2epNcSO

https://colab.research.google.com/github/abhimishra91/transformers-tutorials/blob/master/transformers_multiclass_classification.ipynb#scrollTo=hZtTYVYGwFas

In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.1 MB/s eta 0:00:00


In [7]:
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer, BertTokenizer

https://towardsdatascience.com/transformers-for-multilabel-classification-71a1a0daf5e1

BERT:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) 

XLNet:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=False) 

RoBERTa:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=False)

|Model|#params|Language|
|--|--|--|
|bert-base-uncased|110M|English|
|bert-large-uncased|340M|English|
|bert-base-cased|110M|English|
|bert-large-cased|340M|English|
|bert-base-chinese|110M|Chinese|
|bert-base-multilingual-cased|110M|Multiple|
|bert-large-uncased-whole-word-masking|340M|English|
|bert-large-cased-whole-word-masking|340M|English|


In [18]:
PRETRAINED_MODEL_NAME = 'bert-base-cased'  # 指定BERT-BASE預訓練模型

# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print('PyTorch 版本：', torch.__version__)

PyTorch 版本： 2.0.0+cu118


In [19]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [20]:
# BERT:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) 
# XLNet:
# tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=False) 
# RoBERTa:
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=False)

In [21]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
STEP_NUM = 100
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [44]:
class Triage(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
    self.len = len(dataframe)
    self.data = dataframe
    self.tokenizer = tokenizer
    self.max_len = max_len
    
  def __getitem__(self, index):
    title = str(self.data.ID[index+1])
    title = " ".join(title.split())
    inputs = self.tokenizer.encode_plus(
        title,
        None,
        add_special_tokens=True,
        max_length=self.max_len,
        pad_to_max_length=True,
        return_token_type_ids=True,
        truncation=True
    )
    ids = inputs['input_ids']
    mask = inputs['attention_mask']

    return {
        'ids': torch.tensor(ids, dtype=torch.long),
        'mask': torch.tensor(mask, dtype=torch.long),
        'targets': torch.tensor(self.data.Disease_category[index+1], dtype=torch.long)
    } 
  def __len__(self):
    return self.len

In [45]:
# Creating the dataset and dataloader for the neural network
training_set = Triage(training_data, tokenizer, MAX_LEN)
testing_set = Triage(test_data, tokenizer, MAX_LEN)

In [46]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [47]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [48]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistillBERTClass(torch.nn.Module):
  def __init__(self):
    super(DistillBERTClass, self).__init__()
    self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
    self.pre_classifier = torch.nn.Linear(768, 768)
    self.dropout = torch.nn.Dropout(0.3)
    self.classifier = torch.nn.Linear(768, 5)

  def forward(self, input_ids, attention_mask):
    output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
    hidden_state = output_1[0]
    pooler = hidden_state[:, 0]
    pooler = self.pre_classifier(pooler)
    pooler = torch.nn.ReLU()(pooler)
    pooler = self.dropout(pooler)
    output = self.classifier(pooler)
    return output

In [49]:
model = DistillBERTClass()
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

In [50]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [51]:
# Function to calcuate the accuracy of the model

def calcuate_accu(big_idx, targets):
  n_correct = (big_idx==targets).sum().item()
  return n_correct

訓練

In [52]:
def train(epoch,step_num):
  tr_loss = 0
  n_correct = 0
  nb_tr_steps = 0
  nb_tr_examples = 0
  model.train()
  for _,data in enumerate(training_loader, 0):
    ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)
    targets = data['targets'].to(device, dtype = torch.long)

    outputs = model(ids, mask)
    loss = loss_function(outputs, targets)
    tr_loss += loss.item()
    big_val, big_idx = torch.max(outputs.data, dim=1)
    n_correct += calcuate_accu(big_idx, targets)

    nb_tr_steps += 1
    nb_tr_examples+=targets.size(0)
    
    if _%step_num==0:
      loss_step = tr_loss/nb_tr_steps
      accu_step = (n_correct*100)/nb_tr_examples 
      print(f"Training Loss per {step_num} steps: {loss_step}")
      print(f"Training Accuracy per {step_num} steps: {accu_step}")

    optimizer.zero_grad()
    loss.backward()
    # # When using GPU
    optimizer.step()

  print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
  epoch_loss = tr_loss/nb_tr_steps
  epoch_accu = (n_correct*100)/nb_tr_examples
  print(f"Training Loss Epoch: {epoch_loss}")
  print(f"Training Accuracy Epoch: {epoch_accu}")

  return 

In [53]:
for epoch in range(EPOCHS):
  train(epoch, STEP_NUM)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 100 steps: 1.591320276260376
Training Accuracy per 100 steps: 50.0


TypeError: ignored

驗證

In [ ]:
def valid(model, testing_loader):
  model.eval()
  n_correct = 0; n_wrong = 0; total = 0
  with torch.no_grad():
    for _, data in enumerate(testing_loader, 0):
      ids = data['ids'].to(device, dtype = torch.long)
      mask = data['mask'].to(device, dtype = torch.long)
      targets = data['targets'].to(device, dtype = torch.long)
      outputs = model(ids, mask).squeeze()
      loss = loss_function(outputs, targets)
      tr_loss += loss.item()
      big_val, big_idx = torch.max(outputs.data, dim=1)
      n_correct += calcuate_accu(big_idx, targets)

      nb_tr_steps += 1
      nb_tr_examples+=targets.size(0)
      
      if _%5000==0:
          loss_step = tr_loss/nb_tr_steps
          accu_step = (n_correct*100)/nb_tr_examples
          print(f"Validation Loss per 100 steps: {loss_step}")
          print(f"Validation Accuracy per 100 steps: {accu_step}")
  epoch_loss = tr_loss/nb_tr_steps
  epoch_accu = (n_correct*100)/nb_tr_examples
  print(f"Validation Loss Epoch: {epoch_loss}")
  print(f"Validation Accuracy Epoch: {epoch_accu}")
  
  return epoch_accu


In [ ]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

刪除不需要的欄位

In [ ]:
# 刪除不需要的欄位
x_train_drop=x_train.drop(['heartburn', 'Diabetes', 'Hypertension', 'CAD', 'Head injury', 'CVA'],axis=1)
x_test_drop=x_test.drop(['heartburn', 'Diabetes', 'Hypertension', 'CAD', 'Head injury', 'CVA'],axis=1)

In [ ]:
x_train_drop.columns

儲存模型

In [ ]:
# # Saving the files for re-use

# output_model_file = './models/pytorch_distilbert_news.bin'
# output_vocab_file = './models/vocab_distilbert_news.bin'

# model_to_save = model
# torch.save(model_to_save, output_model_file)
# tokenizer.save_vocabulary(output_vocab_file)

# print('All files saved')
# print('This tutorial is completed')